## Daycon - 상점 총 매출 예측(Store total sales forecast) 

- 한국의 약 2,000개 상점의 신용카드 거래 내역이 제공됩니다. 
- card_id를 기준으로 샘플링되었으며, amount의 단위는 KRW가 아닙니다. 테스트 파일에서 각 상점의 마지막 매출 발생일 다음 날부터 100일 후까지 매출의 총합을 예측해야 합니다.
- 예측한 금액만큼 100일간 대출을 진행합니다. 만약 예측한 금액보다 실제 매출이 적게 발생한다면 원금에 손실을 보게 됩니다. 실제 매출이 예측한 금액 이상으로 발생한다면, 연이율 13%의 이자가 발생합니다.


## Data
- train.csv - 카드 매출 내역, 2016-08-01부터 2018-07-31까지.
- test.csv - train.csv랑 같은 형식. test.csv랑 train.csv의 store_id가 같아도 같은 상점인 것은 아님. 
- (월에 최소 5일 이상 데이터가 있는 상점만을 문제에 포함.)
- submission.csv – store_id와 total_sales 두 개의 열로 구성. store_id는 test.csv의 store_id와 대응.

In [176]:
# Daycon - Store total sales forecast

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O

# 1.데이터 로딩
train = pd.read_csv('C:/Users/urse/Desktop/dataset/Daycon-store_total_sales_forecast/train/train.csv')
test = pd.read_csv('C:/Users/urse/Desktop/dataset/Daycon-store_total_sales_forecast/test/test.csv')
submission = pd.read_csv('C:/Users/urse/Desktop/dataset/Daycon-store_total_sales_forecast/submission.csv')

print('데이터 로딩 완료')
print('train:', train.shape)
print('test:', test.shape)

데이터 로딩 완료
train: (3362796, 8)
test: (473392, 8)


In [177]:
train.head()

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-12-14,18:05:31,d297bba73f,5,NaN,2,0
1,0,2016-12-14,18:05:54,d297bba73f,-5,NaN,2,0
2,0,2016-12-19,12:42:31,0880849c05,144,NaN,0,0
3,0,2016-12-19,12:48:08,8b4f9e0e95,66,NaN,0,0
4,0,2016-12-19,13:31:08,7ad237eed0,24,NaN,0,0


In [178]:
test.head()

,store_id,date,time,card_id,amount,installments,days_of_week,holyday
0,0,2016-08-01,00:28:15,bf33518373,125,NaN,0,0
1,0,2016-08-01,01:09:58,7a19a3a92f,90,NaN,0,0
2,0,2016-08-01,01:47:24,6f9fd7e241,150,NaN,0,0
3,0,2016-08-01,17:54:43,8bcf1d61b2,362,NaN,0,0
4,0,2016-08-01,18:48:53,6a722ce674,125,NaN,0,0


### 2. 데이터 자료들 분석 - train
- 전체 행 갯수 3362796 rows / 8 cols
- store_id - 갯수 : 1775개  범위 : 0~1799 까지
- date - 갯수 : 730개  범위 : 2016-08-01 ~ 2018-07-31(총 2년치)
- time - 갯수 : 85924개  범위 : 00:00:00 ~ 23:59:59 (원래는 86400 모든 시간이 다있는건 아님 )
- amount - 갯수 : 6031개  범위 : -250000 ~ 250000 
- card_id - 갯수 : 85924개 
- installments - 갯수 : 17개  범위 : 2 ~ 36, nan
- days_of_week - 7개  범위 : 0~6
- holyday - 2개  범위 : 0~1

In [179]:
# 데이터 자료 분석 - train
for i in train.columns:
    train_unique = train[i].unique()
    train_unique.sort()
    print(i,':',train_unique ,'/', len(train_unique), '개')

store_id : [   0    1    2 ... 1797 1798 1799] / 1775 개
date : ['2016-08-01' '2016-08-02' '2016-08-03' '2016-08-04' '2016-08-05'
 '2016-08-06' '2016-08-07' '2016-08-08' '2016-08-09' '2016-08-10'
 '2016-08-11' '2016-08-12' '2016-08-13' '2016-08-14' '2016-08-15'
 '2016-08-16' '2016-08-17' '2016-08-18' '2016-08-19' '2016-08-20'
 '2016-08-21' '2016-08-22' '2016-08-23' '2016-08-24' '2016-08-25'
 '2016-08-26' '2016-08-27' '2016-08-28' '2016-08-29' '2016-08-30'
 '2016-08-31' '2016-09-01' '2016-09-02' '2016-09-03' '2016-09-04'
 '2016-09-05' '2016-09-06' '2016-09-07' '2016-09-08' '2016-09-09'
 '2016-09-10' '2016-09-11' '2016-09-12' '2016-09-13' '2016-09-14'
 '2016-09-15' '2016-09-16' '2016-09-17' '2016-09-18' '2016-09-19'
 '2016-09-20' '2016-09-21' '2016-09-22' '2016-09-23' '2016-09-24'
 '2016-09-25' '2016-09-26' '2016-09-27' '2016-09-28' '2016-09-29'
 '2016-09-30' '2016-10-01' '2016-10-02' '2016-10-03' '2016-10-04'
 '2016-10-05' '2016-10-06' '2016-10-07' '2016-10-08' '2016-10-09'
 '2016-10-10'

time : ['00:00:00' '00:00:01' '00:00:02' ... '23:59:57' '23:59:58' '23:59:59'] / 85924 개
card_id : ['000009e765' '00000f9eaa' '0000134335' ... 'ffffe10b92' 'fffffa0825'
 'ffffffab49'] / 922522 개
amount : [-250000 -135980 -133162 ...  135980  220000  250000] / 6031 개
installments : [ 2.  3.  4.  5.  6.  7.  8.  9. 10. 12. 15. 18. 20. 22. 24. 36. nan] / 17 개
days_of_week : [0 1 2 3 4 5 6] / 7 개
holyday : [0 1] / 2 개


### 2. 데이터 자료들 분석 - test data
- 전체 행 갯수 473392 rows / 8 cols
- store_id - 갯수 : 200개  범위 : 0~199 까지
- date - 갯수 : 608개  범위 : 2016-08-01 ~ 2018-03-31
- time - 갯수 : 72181개  범위 : 00:00:00 ~ 23:59:59 (원래는 86400 모든 시간이 다있는건 아님 )
- amount - 갯수 : 2809개  범위 : -25955 ~ 2809 
- card_id - 갯수 : 214584개 
- installments - 갯수 : 12개  범위 : 2 ~ 24, nan
- days_of_week - 7개  범위 : 0~6
- holyday - 2개  범위 : 0~1

In [180]:
# 데이터 자료 분석 - test
for i in test.columns:
    test_unique = test[i].unique()
    test_unique.sort()
    print(i,':',test_unique ,'/', len(test_unique), '개')

store_id : [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199] / 200 개
date : ['2016-08-01' '2016-08-02' '2016-08-03' '2016-08-04' '2016-08-05'
 '2016-08-06' '2016-08-07' '2016-08-08' '2016-08-09' '2016-08-10'
 '2016-08-11' '2016-08-12' '2

time : ['00:00:00' '00:00:01' '00:00:02' ... '23:59:57' '23:59:58' '23:59:59'] / 72181 개
card_id : ['000043b1cb' '000055e420' '0000ed4415' ... 'ffff8c2562' 'ffff951446'
 'ffffa16c42'] / 214584 개
amount : [-25955 -16350 -12510 ...  27970  28750 100000] / 2809 개
installments : [ 2.  3.  4.  5.  6.  7.  8. 10. 12. 15. 24. nan] / 12 개
days_of_week : [0 1 2 3 4 5 6] / 7 개
holyday : [0 1] / 2 개


In [181]:
# 3. 전처리(1) Drop
# 필요없는 time, installments, days_of_week, card_id, holyday 삭제
train_drop = train.drop(columns=['time', 'installments', 'days_of_week', 'card_id', 'holyday'])
test_drop = test.drop(columns=['time', 'installments', 'days_of_week', 'card_id', 'holyday'])

In [182]:
print(train_drop.shape)
train_drop.head()

(3362796, 3)


,store_id,date,amount
0,0,2016-12-14,5
1,0,2016-12-14,-5
2,0,2016-12-19,144
3,0,2016-12-19,66
4,0,2016-12-19,24


In [183]:
print(test_drop.shape)
test_drop.head()

(473392, 3)


,store_id,date,amount
0,0,2016-08-01,125
1,0,2016-08-01,90
2,0,2016-08-01,150
3,0,2016-08-01,362
4,0,2016-08-01,125


In [184]:
# 4. 전처리(2) - 날짜 및 store로 그룹짓고 합치기.
# groupby - agg 구문 : amount의 값들을 같은날짜, 같은 store_id 로 합계를 구함.
# set_index : 기존의 행 인덱스를 제거하고 데이터 열 중 하나를 인덱스로 설정
# reset_index : 기존의 행 인덱스를 제거하고 인덱스를 데이터 열로 추가 --> 

train_agg = train_drop.groupby(['date', 'store_id']).agg({'amount':'sum'}).reset_index()
test_agg = test_drop.groupby(['date', 'store_id']).agg({'amount':'sum'}).reset_index()

# 4. 전처리(3) - 날짜 (date) 변수 date time 형태로 형변환
# infer_datetime_format : 추론할수있는 데이터 형식인지 시도해보고 성공시 5~10%정도 더 빠름
train_agg['date'] = pd.to_datetime(train_agg['date'],infer_datetime_format=True) 
test_agg['date'] = pd.to_datetime(test_agg['date'],infer_datetime_format=True)

In [185]:
# 5. 전처리(3) - 날짜 (date) 변수 date time 형태로 형변환
# infer_datetime_format : 추론할수있는 데이터 형식인지 시도해보고 성공시 5~10%정도 더 빠름
train_agg['date'] = pd.to_datetime(train_agg['date'],infer_datetime_format=True) 
test_agg['date'] = pd.to_datetime(test_agg['date'],infer_datetime_format=True)

In [186]:
# 6. 전처리(4) - temp_date 생성 
train_agg['temp_date'] = train_agg['date']
test_agg['temp_date'] = test_agg['date']

In [187]:
print(train_agg.shape)
train_agg.head()

(480160, 4)


,date,store_id,amount,temp_date
0,2016-08-01,2,2372,2016-08-01
1,2016-08-01,6,1080,2016-08-01
2,2016-08-01,8,3650,2016-08-01
3,2016-08-01,12,1155,2016-08-01
4,2016-08-01,14,1270,2016-08-01


In [188]:
print(test_agg.shape)
test_agg.head()

(68977, 4)


,date,store_id,amount,temp_date
0,2016-08-01,0,2106,2016-08-01
1,2016-08-01,2,647,2016-08-01
2,2016-08-01,3,199,2016-08-01
3,2016-08-01,4,279,2016-08-01
4,2016-08-01,6,718,2016-08-01


In [189]:
# 7. 전처리 (5) - date를 index로
# set_index : 기존의 행 인덱스를 제거하고 데이터 열 중 하나를 인덱스로 설정
# reset_index : 기존의 행 인덱스를 제거하고 인덱스를 데이터 열로 추가 --> 
train_index = train_agg.set_index("date",inplace=False) # inplace_true = 기존 객체를 수정한다는 뜻
test_index = test_agg.set_index("date",inplace=False)

In [190]:
print(train_index.shape)
train_index.head()

(480160, 3)


,store_id,amount,temp_date
date,,,
2016-08-01,2,2372,2016-08-01
2016-08-01,6,1080,2016-08-01
2016-08-01,8,3650,2016-08-01
2016-08-01,12,1155,2016-08-01
2016-08-01,14,1270,2016-08-01


In [191]:
print(test_index.shape)
test_index.head()

(68977, 3)


,store_id,amount,temp_date
date,,,
2016-08-01,0,2106,2016-08-01
2016-08-01,2,647,2016-08-01
2016-08-01,3,199,2016-08-01
2016-08-01,4,279,2016-08-01
2016-08-01,6,718,2016-08-01


### 8. 데이터 전처리 - (6) 가게별 영업 일자 수 체크 및 불필요 가게 drop
- 목적은 향후 100일의 판매량을 예측하는 것이기 떄문에
- store_id의 160일 미만인 것들은 모두 제거.
- 160은 여러 번의 시행착오를 통해 얻은 이상적인 표준 숫자이고 Train 100 test 60으로 실험 예정

In [192]:
# 데이터 자료 분석
for i in train_index.columns:
    train_unique2 = train_index[i].unique()
    train_unique2.sort()
    print(i,':', len(train_unique2), '개')
    
print(type(train_index[i].unique()))

store_id : 1775 개
amount : 10971 개
temp_date : 730 개
<class 'numpy.ndarray'>


In [193]:
# 8. 데이터 전처리 - (6) 가게별 영업 일자 수 체크 및 불필요 가게 drop
# 모든 가게 영업일 수 세기 (1800개의 가게)
# 160 보다 많으면 counter 증가

print('영업일 160일 미만 가게 Drop전 데이터 :', train_index.shape)
limit = 160 # 최소 영업일

store_id_list = train_index['store_id'].unique()
store_id_list.sort()

# 날짜가 160개 미만인 store data
store_less_than_160_date = []
store_more_than_160_date = []
for x in store_id_list:
    # store_id 각각의 날짜데이터 갯수
    date_count = train_index[train_index['store_id']==x]['store_id'].count() 
    
    # 날짜 데이터 갯수가 160개 이상일 때
    if date_count>=limit:
        #print('이상:',x, date_count)
        store_more_than_160_date.append(x)
        
    # 날짜 데이터 갯수가 160개 미만일 때
    else:
        #print('미만:', x, date_count)
        store_less_than_160_date.append(x)
        train_index = train_index[train_index.store_id !=x]

train_drop_store = train_index

# 매출일자가 160일 이상인 가게들만 출력 -> 997개의 가게만 남음
print('영업일 160일 미만 가게 {}'.format(len(store_less_than_160_date)))
print('영업일 160일 이상 가게 {}'.format(len(store_more_than_160_date)))
print('가게 Drop 후 데이터', train_drop_store.shape)

영업일 160일 미만 가게 Drop전 데이터 : (480160, 3)
영업일 160일 미만 가게 778
영업일 160일 이상 가게 997
가게 Drop 후 데이터 (432671, 3)


In [194]:
# 9. 데이터 전처리 - (7) 영업 중이 아닌 상점 훈련데이터 세트에서 제거

# 2018-07-31 이전의 5 일 이내에 상점에 데이터가없는 경우 폐쇄되었다고 가정하고 제거.

from datetime import datetime

def keep_alive_store(df, data_type):    
    store_id_list2 = df.store_id.unique() # 0~1799 중에서 997개

    # yes - 5일 혹은 7일 이내 데이터가 있을 때 / no - 5일 이내 데이터 없음.
    yes, no = 0, 0 
    
    if data_type=='train':
        day_time = 5
        # strptime(date_string, format) / 주어진 해당 포맷으로 문자열을 datetime 으로
        t2 = datetime.strptime('2018-07-31 00:00:00', "%Y-%m-%d %H:%M:%S")
    
    elif data_type == 'test':
        day_time = 7
        # strptime(date_string, format) / 주어진 해당 포맷으로 문자열을 datetime 으로
        t2 = datetime.strptime('2018-03-31 00:00:00', "%Y-%m-%d %H:%M:%S")
    
    # store_id로 검색하는데 temp_data는 date형태여서
    for s in store_id_list2:
        # iloc (series 형으로 변환) -> iloc[-1] 은 맨 마지막 시간 -> 
        # 'temp_date'는 timestamp형 -> 00:00:00 인 이유는 날짜만 적혀있으니까 원본 데이터에
        
        # 맨 마지막 날 시간 데이터 (iloc[-1]로 dataframe 마지막 series 접근)
        time_date = df[df.store_id == s].iloc[-1]['temp_date']
        if data_type =='train':
            # 마지막날 장사를 했다는 것.
            if str(time_date) == '2018-07-31 00:00:00':
                yes +=1

            # 마지막날 장사 데이터가 없다면 
            else:
                t1 = datetime.strptime(str(time_date), "%Y-%m-%d %H:%M:%S")
                time_difference = t2-t1

                if time_difference.days <=day_time: # 사실 위에 시간을 2018-07-26 이런식으로 해도 됨.
                    yes +=1
                else:
                    no +=1
                    df = df[df.store_id !=s] # df에서 데이터 빼기
        else:
            if str(time_date) == '2018-03-31 00:00:00':
                yes +=1

            # 마지막날 장사 데이터가 없다면 
            else:
                t1 = datetime.strptime(str(time_date), "%Y-%m-%d %H:%M:%S")
                time_difference = t2-t1

                if time_difference.days <=day_time: # 사실 위에 시간을 2018-07-26 이런식으로 해도 됨.
                    yes +=1
                else:
                    no +=1
                    df = df[df.store_id !=s] # df에서 데이터 빼기            
    print(day_time,'일 이상 장사를 안한 가게 수 - ', no, '장사를 하고 있는 가게 수 - ', yes)
    return df

# print(train_drop_store.values) # temp_data의 형이 timestamp형인 것을 알 수 있음.
train_alive_store = keep_alive_store(train_drop_store, 'train')
print('train_alive_store', train_alive_store.shape)

5 일 이상 장사를 안한 가게 수 -  91 장사를 하고 있는 가게 수 -  906
train_alive_store (400480, 3)


In [195]:
# 10. 데이터 전처리 - (8) 영업 중이 아닌 상점 테스트데이터 제거
# 2018-03-31 부터 7일 이내에 상점에 데이터가 없는 경우, 폐쇄 된 것으로 가정 (2018-03-31 이 마지막 날짜)

print(test_index.shape)
test_alive_store = keep_alive_store(test_index, 'test')
print(test_alive_store.shape)

(68977, 3)
7 일 이상 장사를 안한 가게 수 -  4 장사를 하고 있는 가게 수 -  196
(68207, 3)


In [196]:
# 11. 데이터 전처리 - (9) 
# 각 store_id를 새로운 데이터 프레임으로 그룹화 시 누락된 날짜는 0으로 채워짐
# 이동평균, y값 (미래 100일), 평균, 중앙값, 합계 등 새로운 열이 추가됨.

def reform_data(df, isTrain):
    store_id_list = df.store_id.unique() # list of store_idw
    x_array = [] # array to return

    for s in store_id_list: # iterate through each store_id
        store = df[df.store_id == s]
        
        # Filling missing dates with value of 0
        store = store.asfreq('D', fill_value=0)
        store['temp_date'] = store.index
        store['store_id'] = s
        
        # Moving Average columns were added
        store['MA7'] = store['amount'].rolling('7D').mean() # rolling = 순차적으로 평균내는 것
        store['MA15'] = store['amount'].rolling('15D').mean() # rolling = 순차적으로 평균내는 것
        store['MA30'] = store['amount'].rolling('30D').mean() # rolling = 순차적으로 평균내는 것

        # 훈련 데이터일때
        # 100일까지 기록하고 나머지는 store_x에 저장하여 
        if isTrain:
            store_y = store.last('100D') # last 100 days of store data
            y = store_y.amount.sum()
            store_x = store[store.temp_date < store_y.iloc[0].temp_date] # data except last 100 days
            
        # 테스트 데이터일때
        # For the test dataframe, all data was kept as 'store_x'.
        # Of course the 'y' is 0 here because that's to be predicted later on.
        else:
            y = 0
            store_x = store[:]
            
        new_data = [] # array for each store's new data
        new_data.append(s) # store_id
        new_data.append(y) # total sum of last 100 days (answer)

        new_data.append(store_x.amount.mean()) # mean of amount
        new_data.append(store_x.amount.median()) # median of amount
        
        new_data.append(store_x.last('7D').amount.mean()) # mean of amount (last 7 days)
        new_data.append(store_x.last('15D').amount.mean()) # mean of amount (last 15 days)
        new_data.append(store_x.last('30D').amount.mean()) # mean of amount (last 30 days)
        
        new_data.append(store_x.last('7D').amount.median()) # median of amount (last 7 days)
        new_data.append(store_x.last('15D').amount.median()) # median of amount (last 15 days)
        new_data.append(store_x.last('30D').amount.median()) # median of amount (last 30 days)
        
        new_data.append(store_x.last('7D').amount.sum()) # sum of amount (last 7 days)
        new_data.append(store_x.last('15D').amount.sum()) # sum of amount (last 15 days)
        new_data.append(store_x.last('30D').amount.sum()) # sum of amount (last 30 days)
        
        new_data.append(store_x.last('7D').MA7.mean()) # mean of Moving Average of 7D (last 7 days)
        new_data.append(store_x.last('15D').MA7.mean()) # mean of Moving Average of 7D (last 15 days)
        new_data.append(store_x.last('30D').MA7.mean()) # mean of Moving Average of 7D (last 30 days)
        new_data.append(store_x.last('7D').MA15.mean()) # mean of Moving Average of 15D (last 7 days)
        new_data.append(store_x.last('15D').MA15.mean()) # mean of Moving Average of 15D (last 15 days)
        new_data.append(store_x.last('30D').MA15.mean()) # mean of Moving Average of 15D (last 30 days)
        new_data.append(store_x.last('7D').MA30.mean()) # mean of Moving Average of 30D (last 7 days)
        new_data.append(store_x.last('15D').MA30.mean()) # mean of Moving Average of 30D (last 15 days)
        new_data.append(store_x.last('30D').MA30.mean()) # mean of Moving Average of 30D (last 30 days)
        
        x_array.append(new_data) # Append the 'new_data' array in to 'x_array'
    if isTrain:
        print('Completed reform train')
    else:
        print("Completed reform test")
    return x_array

# reformed_train, reforemd_test => list 형태
reformed_train = reform_data(train_alive_store, True) # train data with new values
reformed_test = reform_data(test_alive_store, False) # test data with new values



Completed reform train
Completed reform test


In [197]:
# Now the returned array is back to its dataframe form with columns names.
# t => train / r_test => test
t = pd.DataFrame(reformed_train, columns=['store_id', 'y', 'mean', 'median', '7mean', '15mean', '30mean', 
                                         '7median', '15median', '30median',  '7sum', '15sum', '30sum', 
                                         '7ma7mean', '15ma7mean', '30ma7mean',  '7ma15mean', '15ma15mean',
                                         '30ma15mean',  '7ma30mean', '15ma30mean', '30ma30mean'])
r_test = pd.DataFrame(reformed_test, columns=['store_id', 'y', 'mean', 'median', '7mean', '15mean', '30mean', 
                                         '7median', '15median', '30median',  '7sum', '15sum', '30sum', 
                                         '7ma7mean', '15ma7mean', '30ma7mean',  '7ma15mean', '15ma15mean',
                                         '30ma15mean',  '7ma30mean', '15ma30mean', '30ma30mean'])

In [198]:
print('train_shape', t.shape)
t.head()

train_shape (906, 22)


,store_id,y,mean,median,7mean,15mean,30mean,7median,15median,30median,...,30sum,7ma7mean,15ma7mean,30ma7mean,7ma15mean,15ma15mean,30ma15mean,7ma30mean,15ma30mean,30ma30mean
0,2,186853,2266.995238,2267.0,2207.714286,2209.466667,2272.233333,2174.0,2174.0,2293.0,...,68167,2201.020408,2203.819048,2297.928571,2203.819048,2233.044444,2319.146667,2297.928571,2319.146667,2344.041111
1,6,219761,1857.495238,1080.0,2076.428571,2359.133333,1970.233333,1670.0,1670.0,1622.5,...,59107,2703.020408,2448.704762,2690.871429,2448.704762,2061.066667,2833.988889,2690.871429,2833.988889,3410.305556
2,8,79695,691.809524,500.0,623.571429,704.333333,650.666667,260.0,450.0,475.0,...,19520,570.102041,659.047619,591.166667,659.047619,635.622222,629.444444,591.166667,629.444444,664.450000
3,12,147744,1224.731746,898.5,1995.000000,1524.333333,1118.200000,2145.0,1155.0,990.0,...,33546,1772.346939,1273.571429,971.723810,1273.571429,990.880000,860.555556,971.723810,860.555556,824.680000
4,14,17253,309.682035,150.0,267.142857,218.733333,340.766667,195.0,187.0,153.5,...,10223,252.836735,471.485714,341.904762,471.485714,465.400000,346.037778,341.904762,346.037778,305.583333


In [199]:
print('test_shape', r_test.shape)
r_test.head()

test_shape (196, 22)


,store_id,y,mean,median,7mean,15mean,30mean,7median,15median,30median,...,30sum,7ma7mean,15ma7mean,30ma7mean,7ma15mean,15ma15mean,30ma15mean,7ma30mean,15ma30mean,30ma30mean
0,0,0,1049.764803,981.5,886.142857,1028.666667,1068.800000,821.0,877.0,929.5,...,32064,1096.183673,1057.038095,1109.019048,1057.038095,1096.680000,1119.191111,1109.019048,1119.191111,1098.122222
1,2,0,474.812191,170.0,145.000000,172.666667,337.166667,0.0,125.0,122.5,...,10115,140.510204,151.238095,328.857143,151.238095,281.844444,349.700000,328.857143,349.700000,398.294444
2,3,0,377.111842,328.5,439.571429,597.266667,471.866667,409.0,409.0,359.0,...,14156,456.020408,621.533333,461.619048,621.533333,584.377778,443.811111,461.619048,443.811111,390.116667
3,4,0,260.809211,236.5,246.428571,285.666667,252.100000,193.0,230.0,236.0,...,7563,274.224490,271.228571,238.933333,271.228571,255.768889,226.737778,238.933333,226.737778,204.060000
4,6,0,929.853618,894.5,857.428571,1007.133333,1010.366667,895.0,910.0,1002.0,...,30311,1005.081633,986.819048,1075.100000,986.819048,973.231111,1081.684444,1075.100000,1081.684444,994.248889
